In [1]:
import tensorflow as tf
import numpy as np, h5py
import scipy.io as sio
import sys
import random
import kNN
import re
from numpy import *   
import os

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def compute_accuracy(test_att, test_visual, test_id, test_label):
    global left_a2  
    att_pre = sess.run(left_a2, feed_dict={att_features: test_att})
    test_id = np.squeeze(np.asarray(test_id))
    outpre = [0]*6180  # CUB 2933
    test_label = np.squeeze(np.asarray(test_label))
    test_label = test_label.astype("float32")
    for i in range(6180):  # CUB 2933
        outputLabel = kNN.kNNClassify(test_visual[i,:], att_pre, test_id, 1) 
        outpre[i] = outputLabel
    correct_prediction = tf.equal(outpre, test_label)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={
                                           att_features: test_att, visual_features: test_visual})
    return result

# # data shuffle
def data_iterator():
    """ A simple data iterator """
    batch_idx = 0
    while True:
        # shuffle labels and features
        idxs = np.arange(0, len(x))
        np.random.shuffle(idxs)
        shuf_visual = x[idxs]
        shuf_att = att[idxs]
        batch_size = 64
        for batch_idx in range(0, len(x), batch_size):
            visual_batch = shuf_visual[batch_idx:batch_idx+batch_size]
            visual_batch = visual_batch.astype("float32")
            att_batch = shuf_att[batch_idx:batch_idx+batch_size]
            yield att_batch, visual_batch

os.environ["CUDA_VISIBLE_DEVICES"] = ""

# # data
f=h5py.File('./data/AwA_data/attribute/Z_s_con.mat','r')
att=np.array(f['Z_s_con'])
print(att.shape)

f=sio.loadmat('./data/AwA_data/train_googlenet_bn.mat')
x=np.array(f['train_googlenet_bn'])
print(x.shape)

f=sio.loadmat('./data/AwA_data/test_googlenet_bn.mat')
x_test=np.array(f['test_googlenet_bn'])
print(x_test.shape)

f=sio.loadmat('./data/AwA_data/test_labels.mat')
test_label=np.array(f['test_labels'])
print(test_label.shape)

f=sio.loadmat('./data/AwA_data/testclasses_id.mat')
test_id=np.array(f['testclasses_id'])
print(test_id.shape)

f=sio.loadmat('./data/AwA_data/attribute/pca_te_con_10x85.mat')
att_pro=np.array(f['pca_te_con_10x85'])
print(att_pro.shape)

(24295, 85)
(24295, 1024)
(6180, 1024)
(6180, 1)
(10, 1)
(10, 85)


In [2]:
# # Placeholder
# define placeholder for inputs to network
att_features = tf.placeholder(tf.float32, [None, 85])
visual_features = tf.placeholder(tf.float32, [None, 1024])


# # Network
# AwA 85 300 1024 ReLu, 1e-2 * regularisers, 64 batch, 0.0001 Adam
W_left_a1 = weight_variable([85, 300])
b_left_a1 = bias_variable([300])
left_a1 = tf.nn.relu(tf.matmul(att_features, W_left_a1) + b_left_a1)


W_left_a2 = weight_variable([300, 1024])
b_left_a2 = bias_variable([1024])
left_a2 = tf.nn.relu(tf.matmul(left_a1, W_left_a2) + b_left_a2)


# # loss
loss_a = tf.reduce_mean(tf.square(left_a2 - visual_features))    

# L2 regularisation for the fully connected parameters.
regularisers_a = (tf.nn.l2_loss(W_left_a1) + tf.nn.l2_loss(b_left_a1)
                + tf.nn.l2_loss(W_left_a2) + tf.nn.l2_loss(b_left_a2))

# Add the regularisation term to the loss.            
loss_a += 1e-2 * regularisers_a

train_step = tf.train.AdamOptimizer(0.0001).minimize(loss_a)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


# # Run
iter_ = data_iterator()
for i in range(1000000):
    att_batch_val, visual_batch_val = next(iter_)
    sess.run(train_step, feed_dict={att_features: att_batch_val, visual_features: visual_batch_val})
    if i % 1000 == 0:
        print(compute_accuracy(att_pro, x_test, test_id, test_label))

0.11553398
0.117313914
0.15485437
0.15614887
0.15355986
0.14126213
0.17394821
0.25097087
0.36197412
0.56116503
0.723301
0.7762136
0.8346278
0.8512945
0.8446602
0.84822005
0.84514564
0.84158576
0.83446604
0.84692556
0.8323625
0.8365696
0.8417476
0.831877
0.844822
0.8244337
0.83478963
0.83478963
0.8367314
0.84417474
0.839644
0.818123
0.82378644
0.82281554
0.820712
0.82152104
0.8194175
0.8278317
0.83478963
0.82508093
0.8252427
0.81666666
0.8276699
0.8391586
0.8365696
0.82378644
0.81634307
0.8110032
0.8252427
0.83527505
0.8226537
0.83576053
0.82718444
0.82556635
0.8223301
0.81488675
0.8289644
0.8224919
0.82993525
0.8354369
0.82864076
0.83365697
0.8276699
0.82588995
0.82864076
0.8184466
0.8414239
0.82686085
0.83624595
0.82815534
0.8404531
0.8444984
0.8399676
0.83349514
0.83220065
0.83090615
0.83754045
0.83317155
0.8313916
0.8182848
0.8302589
0.83042073
0.8302589
0.83317155
0.831877
0.8406149
0.8378641
0.842233
0.8420712
0.8407767
0.8365696
0.83446604
0.83899677
0.83721685
0.84029126
0.82993

KeyboardInterrupt: 

In [3]:
f = open(r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/train/attributes_per_class.txt')
lines_attr = f.readlines()
attributes = dict()
for each in lines_attr:
    tokens = each.split()
    #print(tokens)
    label = tokens[0]
    attr_list = list()
    for idx in range(1, 26):
        attr_list.append(float(tokens[idx]))
    if not (len(attr_list) == 25):
        print(('attributes number error\n'))
        exit()
    attributes[label] = attr_list

In [27]:
print(len(attributes.values()))
print(tuple(attributes.values()))
tmp = np.asarray(tuple(attributes.values()))
print(tmp)
print(tmp[190:].shape)

230
([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.1, 0.3, 0.1, 0.3, 0.1, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 

In [18]:
print(len(attributes.values()))

230
